In [ ]:
%use plotly
import kotlinx.coroutines.*
import java.time.Duration
import java.time.Instant

val data = runBlocking { readAllUData("./data/log.anon") + readAllUData("./data/log2.anon") }
val cacheStats = parseQueryLoad("./data/Query-load.csv") + parseQueryLoad("./data/Query-load2.csv")

In [ ]:
fun add0WithinData(toFill: List<UData>): List<Pair<UInt, List<UData>>> {
    val filledQueryPerSecond = mutableListOf<Pair<UInt, List<UData>>>()
    var current = mutableListOf<UData>()
    var currentTime = toFill.first().unixTimeStamp
    toFill.forEach { u ->
        if (u.unixTimeStamp == currentTime) {
            current.add(u)
        } else {
            filledQueryPerSecond.add(Pair(currentTime, current))
            val afterFirst = currentTime + 1u
            val beforeLast = u.unixTimeStamp - 1u
            if (afterFirst == beforeLast) {
                filledQueryPerSecond.add(Pair(afterFirst, listOf()))
            } else if (afterFirst < beforeLast) {
                filledQueryPerSecond.add(Pair(afterFirst, listOf()))
                filledQueryPerSecond.add(Pair(beforeLast, listOf()))
            }
            current = mutableListOf(u)
            currentTime = u.unixTimeStamp
        }
    }
    filledQueryPerSecond.add(Pair(currentTime, current))
    return filledQueryPerSecond
}

val splitPerSuffix = data.groupBy { ud -> ud.domainName }.mapValues { (_, v) -> add0WithinData(v) }

val queryPerSecond = add0WithinData(data)

val offset = Duration.ofHours(2)

fun convertTime(t: UInt): String {
    return Instant.ofEpochSecond(t.toLong()).plus(offset).toString() // Convert to CET
}

fun plotOverTime(
    udata: Map<List<String>, List<Pair<UInt, List<UData>>>>,
    title: String,
    includeSub: Boolean = true,
    cacheStats: List<CacheStat> = listOf(),
    cpuUsage: Boolean = false,
): Plot {

    fun udToCPU(ud: UData): Double {
        // https://bench.cr.yp.to/results-sign.html
        // amd64; Coffee Lake (906ea); 2018 Intel Xeon E-2124; 4 x 3300MHz; r24000, supercop-20230530
        // does not include time of hashing algorithm and size of hash not considered
        val ghz = (3.3 * Math.pow(10.0, 9.0))
        return when (toSigningAlgorithm(ud.algo)) {
            SigningAlgorithm.RSA -> 54453.0 / ghz // Assuming 2048
            SigningAlgorithm.ECDSA_P_256 -> 284798 / ghz
            SigningAlgorithm.ECDSA_P_384 -> 2673102 / ghz
            SigningAlgorithm.Ed25519 -> 164526.0 / ghz
            SigningAlgorithm.Ed448 -> 506961.0 / ghz
        }
    }

    val perSuffixTraces = udata.map { (suffix, v) ->
        Scatter() {
            val values = v.map { (s, udl) -> Pair(s, udl.filter { ud -> includeSub || ud.sub == false }) }
            val yNumbers = values.map { (_, udl) -> udl.size }
            name = suffix.joinToString(".")
            x.strings = values.map { (t, _) -> convertTime(t) }
            y.numbers = yNumbers
        }
    }.sortedByDescending { t -> t.y.numbers.sumOf { n -> n.toInt() } }

    val cpuTrace: Scatter by lazy {
        Scatter() {
            name = "CPU usage"
            yaxis = "y4"
            x.strings = queryPerSecond.map { (t, _) -> convertTime(t) }
            y.numbers = queryPerSecond.map { (_, udl) -> udl.map { ud -> udToCPU(ud) }.sum() * 100 }
            marker {
                color("rgba(44,160,44,1)")
            }
        }
    }

    var todo = 0.0

    val sqiCpuTrace: Scatter by lazy {
        Scatter() {
            name = "CPU usage"
            yaxis = "y3"
            x.strings = queryPerSecond.map { (t, _) -> convertTime(t) }
            y.numbers = queryPerSecond.map { (_, udl) ->
                var currentCPU = todo + udl.map { ud ->
                    if (ud.domainName.size <= 2 && !ud.sub) {
                        6.7 * 0.8106 / 1000
                    } else {
                        udToCPU(ud)
                    }
                }.sum() * 100
                if (currentCPU > 100) {
                    todo = currentCPU - 100
                    currentCPU = 100.0
                } else {
                    todo = 0.0
                }
                currentCPU
            }
            marker {
                color("rgba(255,127,14,1)")
            }
        }
    }

    val amountOfQueriesTrace = Scatter() {
        name = "Requests"
        x.strings = cacheStats.map { cs -> cs.time.toInstant().plus(offset).toString() }
        y.numbers = cacheStats.map { cs -> (cs.cacheHits + cs.cacheMisses) }
        yaxis = "y2"
        marker {
            color("rgba(44,160,44,1)")
        }
    }

    val cacheHitsTrace = Scatter() {
        name = "Cache hits"
        x.strings = cacheStats.map { cs -> cs.time.toInstant().plus(offset).toString() }
        y.numbers = cacheStats.map { cs -> cs.cacheHits }
        yaxis = "y2"
    }

    val cacheMisses = Scatter() {
        name = "Cache misses"
        x.strings = cacheStats.map { cs -> cs.time.toInstant().plus(offset).toString() }
        y.numbers = cacheStats.map { cs -> cs.cacheMisses }
        yaxis = "y2"
    }

    val tracesToPlot = perSuffixTraces.toMutableList()
    if (!cacheStats.isEmpty()) {
        tracesToPlot.add(amountOfQueriesTrace)
    }
    if (cpuUsage) {
        tracesToPlot.add(cpuTrace)
        tracesToPlot.add(sqiCpuTrace)
    }

    return Plotly.plot {
        traces(tracesToPlot)

        layout {
            width = 1700 * 0.5
            height = 950 * 0.5
            title {
                text = "$title"
            }
            xaxis {
                title {
                    text = "Time"
                }
            }
            if (!cpuUsage && cacheStats.isEmpty()) {
                yaxis {
                    title {
                        text = "Amount of validations per second"
                    }
                    type = AxisType.linear
                }
            }
            if (cacheStats.isNotEmpty()) {
                yaxis {
                    title {
                        text = "Validations per second"
                    }
                    color("rgba(31,119,180,1")
                    type = AxisType.linear
                    overlaying = "y2"
                }
                yaxis(2, {
                    title {
                        text = "Requests per second"
                    }
                    side = AxisSide.right
                    color("rgba(44,160,44,1)")
                })
            }
            if (cpuUsage) {
                yaxis {
                    title {
                        text = "Validations per second"
                    }
                    color("rgba(31,119,180,1")
                    type = AxisType.linear
                    range(0.0..120.0)
                }
                showlegend = false
                yaxis(3, {
                    title {
                        text = "CPU usage percentage per second"
                    }
                    side = AxisSide.right
                    color("rgba(255,127,14,1)")
                    range(0.0..100.0)
                    overlaying = "y"
                })
                // 2 axis for cpu percentages can be used to enfore order user 'overlaying'
                yaxis(4, {
                    visible = false
                    range(0.0..100.0)
                    overlaying = "y"
                })
            }
        }
    }
}

val amountTLDs2 =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size <= 2 }) }

plotOverTime(
    mapOf(Pair(listOf("TLDs and root zone"), amountTLDs2)),
    "Amount of validations over time for TLDs and root zone",
    false,
    cpuUsage = true,
)

In [ ]:
plotOverTime(
    mapOf(Pair(listOf("Validations"), queryPerSecond)),
    "Amount of requests and signature validations over time",
    cacheStats = cacheStats
).apply {
    layout {
        showlegend = false
        legend {
            traceorder = TraceOrder.reversed
        }
    }
}

In [ ]:
plotOverTime(mapOf(Pair(listOf("all"), queryPerSecond)), "Amount of validations over time")

In [ ]:
val amountRootZone =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size == 1 }) }

plotOverTime(mapOf(Pair(listOf(), amountRootZone)), "Amount of validations over time for root zone")

In [ ]:
val amountTLDs =
    queryPerSecond.map { (time, data) -> Pair(time, data.filter { ud -> ud.domainName.size <= 2 }) }

plotOverTime(
    mapOf(Pair(listOf("TLDs and root zone"), amountTLDs)),
    "Amount of validations over time for TLDs and root zone",
    false
)

In [ ]:
plotOverTime(splitPerSuffix, "Amount of validations over time per suffix including subdomains")

In [ ]:
plotOverTime(
    splitPerSuffix,
    "Amount of validations over time per suffix excluding subdomains",
    false
)